In [6]:
import os
import cv2
import time
import pickle
import sklearn
import cvlib as cv
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt



from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, GlobalAveragePooling2D
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.applications import ResNet50, InceptionResNetV2, MobileNetV2, VGG16
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import ResNet50,InceptionResNetV2
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from skmultilearn.model_selection import IterativeStratification
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Flatten, Dense
from sklearn.metrics import classification_report 
from tensorflow.keras.applications import VGG16
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from keras.utils import to_categorical


In [10]:
# Function to load the pre-trained soft voting model using pickle
def load_soft_voting_model(model_file_path):
    with open(model_file_path, 'rb') as model_file:
        soft_voting_model = pickle.load(model_file)
    return soft_voting_model['models']


soft_voting_model_file_path = "C:/Users/Project/Desktop/Deep_fake/DeepFake_Video_Detection/DeepFake_Video_Detection/deep_fake-html/soft_voting_results.pkl"
# Load the soft voting model
loaded_soft_voting_model = load_soft_voting_model(soft_voting_model_file_path)

In [18]:
# Function to process a single video, preprocess frames, and classify using a pre-loaded soft voting model
def process_and_classify_video(video_path, frameTime, soft_voting_model):
    ListFrames = []
    video_ids = []
    frame_ids = []

    total_videos = 0

    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(frame_rate * frameTime)

    frame_counter = 0
    video_frame_ids = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_counter += 1

        if frame_counter % frame_interval == 0:
            frame = cv2.resize(frame, (128, 128))
            ListFrames.append(frame)

            video_ids.append(total_videos)
            frame_ids.append(video_frame_ids)
            video_frame_ids += 1

    cap.release()

    data = {
        'VideoID': video_ids,
        'FrameID': frame_ids,
        'Frames': ListFrames,
    }
    df = pd.DataFrame(data)

    print(f"Capture one frame every {frameTime} seconds")
    print(f"Total number of frames: {len(df)}")

    # Preprocess frames
    df['Frames'] = df['Frames'] / 255.0

    # Classify videos based on each model in the loaded models
    unique_videos = df['VideoID'].unique()
    video_results_details = []

    for model_index, model in enumerate(soft_voting_model):
        model_results = []

        for vid in unique_videos:
            video_subset = df[df['VideoID'] == vid]
            X_video = tf.convert_to_tensor(np.array(video_subset['Frames'].tolist()), dtype=tf.float32)

            # Make predictions using the current model
            predictions = model.predict(X_video)

            # Count the occurrences of predicted labels
            count_fake = np.sum(predictions == 0)  # Assuming 0 corresponds to 'Fake' in the model

            # Determine the model decision ('Real' or 'Fake') based on the count of 'Fake' frames
            total_count = len(video_subset)
            if count_fake / total_count >= 0.3:
                result = 'Fake'
            else:
                result = 'Real'

            percentage_fake = (count_fake / total_count) * 100
            model_results.append((vid, result, percentage_fake))

        video_results_details.append(model_results)

    # Apply soft voting to the individual predictions from each model
    soft_voting_results = []

    for i in range(len(unique_videos)):
        video_id = unique_videos[i]
        video_results = [model_results[i] for model_results in video_results_details]

        # Soft voting: sum up the individual predictions for each video
        avg_prediction = np.mean([result[1] == 'Fake' for result in video_results])

        # Determine the soft voting decision ('Real' or 'Fake') based on label proportions
        if avg_prediction >= 0.5:  # Assuming 0.5 as the threshold for 'Fake' in soft voting
            result = 'Fake'
        else:
            result = 'Real'

        confidence_percentage = avg_prediction * 100
        soft_voting_results.append((video_id, result, confidence_percentage))

    # Determine the final result based on the soft voting decision
    final_result = 'Fake' if any(result == 'Fake' for _, result, _ in soft_voting_results) else 'Real'

    # Display the final result with confidence percentage
    print("\nFinal Result:")
    print(f"The video is classified as {final_result} by {confidence_percentage:.2f}% confidence.")



# Example usage
video_path = "C:/Users/Project/Desktop/adaoqecjet.mp4"
frame_time_seconds = 2
# Process and classify the video using the loaded soft voting model
process_and_classify_video(video_path, frame_time_seconds, loaded_soft_voting_model)


Capture one frame every 2 seconds
Total number of frames: 5
1/1 [==============================] - 0s 31ms/step

Final Result:
The video is classified as Real by 0.00% confidence.


In [19]:


# Function to process a single video, preprocess frames, and classify using a pre-loaded soft voting model
def process_and_classify_video(video_path, frameTime, soft_voting_model):
    ListFrames = []
    video_ids = []
    frame_ids = []

    total_videos = 0

    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(frame_rate * frameTime)

    frame_counter = 0
    video_frame_ids = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_counter += 1

        if frame_counter % frame_interval == 0:
            frame = cv2.resize(frame, (128, 128))
            ListFrames.append(frame)

            video_ids.append(total_videos)
            frame_ids.append(video_frame_ids)
            video_frame_ids += 1

    cap.release()

    data = {
        'VideoID': video_ids,
        'FrameID': frame_ids,
        'Frames': ListFrames,
    }
    df = pd.DataFrame(data)

    print(f"Capture one frame every {frameTime} seconds")
    print(f"Total number of frames: {len(df)}")

    # Preprocess frames
    df['Frames'] = df['Frames'] / 255.0

    # Classify videos based on each model in the loaded models
    unique_videos = df['VideoID'].unique()
    video_results_details = []

    for model_index, model in enumerate(soft_voting_model):
        model_results = []

        for vid in unique_videos:
            video_subset = df[df['VideoID'] == vid]
            X_video = tf.convert_to_tensor(np.array(video_subset['Frames'].tolist()), dtype=tf.float32)

            # Make predictions using the current model
            predictions = model.predict(X_video)

            # Count the occurrences of predicted labels
            count_fake = np.sum(predictions == 0)  # Assuming 0 corresponds to 'Fake' in the model

            # Determine the model decision ('Real' or 'Fake') based on the count of 'Fake' frames
            total_count = len(video_subset)
            if count_fake / total_count >= 0.3:
                result = 'Fake'
            else:
                result = 'Real'

            percentage_fake = (count_fake / total_count) * 100
            model_results.append((vid, result, percentage_fake))

        video_results_details.append(model_results)

    # Print individual model decisions
    print("\nIndividual Model Decisions:")
    for model_index, model_results in enumerate(video_results_details):
        print(f"Results for Model {model_index + 1}:")
        for vid, result, percentage_fake in model_results:
            print(f"  Video ID {vid}: {result}, Percentage Fake: {percentage_fake:.2f}%")

    # Apply soft voting to the individual predictions from each model
    soft_voting_results = []

    for i in range(len(unique_videos)):
        video_id = unique_videos[i]
        video_results = [model_results[i] for model_results in video_results_details]

        # Soft voting: sum up the individual predictions for each video
        avg_prediction = np.mean([result[1] == 'Fake' for result in video_results])

        # Determine the soft voting decision ('Real' or 'Fake') based on label proportions
        if avg_prediction >= 0.5:  # Assuming 0.5 as the threshold for 'Fake' in soft voting
            result = 'Fake'
        else:
            result = 'Real'

        confidence_percentage = avg_prediction * 100
        soft_voting_results.append((video_id, result, confidence_percentage))

    # Determine the final result based on the soft voting decision
    final_result = 'Fake' if any(result == 'Fake' for _, result, _ in soft_voting_results) else 'Real'

    # Display the final result with confidence percentage
    print("\nFinal Result:")
    for vid, result, confidence_percentage in soft_voting_results:
        print(f"The video is classified as {result} by {confidence_percentage:.2f}% confidence.")

# Example usage
video_path = "C:/Users/Project/Desktop/adaoqecjet.mp4"
frame_time_seconds = 2
# Load the soft voting model

# Process and classify the video using the loaded soft voting model
process_and_classify_video(video_path, frame_time_seconds, loaded_soft_voting_model)


Capture one frame every 2 seconds
Total number of frames: 5
1/1 [==============================] - 0s 32ms/step

Individual Model Decisions:
Results for Model 1:
  Video ID 0: Real, Percentage Fake: 0.00%
Results for Model 2:
  Video ID 0: Real, Percentage Fake: 0.00%
Results for Model 3:
  Video ID 0: Real, Percentage Fake: 0.00%

Final Result:
The video is classified as Real by 0.00% confidence.


In [20]:
# Function to process a single video, preprocess frames, and classify using a pre-loaded soft voting model
def process_and_classify_video(video_path, frameTime, soft_voting_model):
    ListFrames = []
    video_ids = []
    frame_ids = []

    total_videos = 0

    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(frame_rate * frameTime)

    frame_counter = 0
    video_frame_ids = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_counter += 1

        if frame_counter % frame_interval == 0:
            frame = cv2.resize(frame, (128, 128))
            ListFrames.append(frame)

            video_ids.append(total_videos)
            frame_ids.append(video_frame_ids)
            video_frame_ids += 1

    cap.release()

    data = {
        'VideoID': video_ids,
        'FrameID': frame_ids,
        'Frames': ListFrames,
    }
    df = pd.DataFrame(data)

    print(f"Capture one frame every {frameTime} seconds")
    print(f"Total number of frames: {len(df)}")

    # Preprocess frames
    df['Frames'] = df['Frames'] / 255.0

    # Classify videos based on each model in the loaded models
    unique_videos = df['VideoID'].unique()
    video_results_details = []

    for model_index, model in enumerate(soft_voting_model):
        model_results = []

        for vid in unique_videos:
            video_subset = df[df['VideoID'] == vid]
            X_video = tf.convert_to_tensor(np.array(video_subset['Frames'].tolist()), dtype=tf.float32)

            # Make predictions using the current model
            predictions = model.predict(X_video)

            # Count the occurrences of predicted labels
            count_real = np.sum(predictions == 1)  # Assuming 1 corresponds to 'Real' in the model

            # Determine the model decision ('Real' or 'Fake') based on the count of 'Real' frames
            total_count = len(video_subset)
            percentage_real = (count_real / total_count) * 100
            model_results.append((vid, percentage_real))

        video_results_details.append(model_results)

    # Apply soft voting to the individual predictions from each model
    soft_voting_results = []

    for i in range(len(unique_videos)):
        video_id = unique_videos[i]
        video_results = [model_results[i] for model_results in video_results_details]

        # Soft voting: sum up the individual predictions for each video
        avg_prediction = np.mean([result[1] for result in video_results])  # Assuming index 1 corresponds to 'Real' percentage

        # Determine the soft voting decision ('Real' or 'Fake') based on label proportions
        if avg_prediction >= 50.0:  # Assuming 50.0 as the threshold for 'Real' in soft voting
            result = 'Real'
        else:
            result = 'Fake'

        confidence_percentage = avg_prediction

        soft_voting_results.append((video_id, result, confidence_percentage))

    # Display the final result with confidence percentage
    print("\nFinal Result:")
    for vid, result, confidence_percentage in soft_voting_results:
        print(f"The video is classified as {result} by {confidence_percentage:.2f}% confidence.")


# Example usage
video_path = "C:/Users/Project/Desktop/adaoqecjet.mp4"
frame_time_seconds = 2
# Process and classify the video using the loaded soft voting model
process_and_classify_video(video_path, frame_time_seconds, loaded_soft_voting_model)


Capture one frame every 2 seconds
Total number of frames: 5
1/1 [==============================] - 0s 33ms/step

Final Result:
The video is classified as Fake by 0.00% confidence.
